## SSPI Regression Analysis

In [232]:
import numpy as np,math
import pandas as pd
import math

from sklearn.experimental import enable_iterative_imputer  
#from sklearn.impute import IterativeImputer

from linearmodels.panel import PanelOLS

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import stats
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.diagnostic import acorr_ljungbox


### Regression Analysis of SSPI Against GDP

Requirements: \
The correlation analysis for the 2018 Static SSPI was very simple because we had one year of data. With panel data, we must be much more careful with our regression analysis. In particular, we need to contend with temporal autocorrelation (within a country, adjacent observations are correlated) and cross-sectional dependence (different countries are different, which causes clustering by country). - There are many methods to deal with these problems in econometrics: fixed effects models, cluster robust panel regression (fixed effects + clustered standard errors), Arellano Method, Driscoll-Kraay Method, Panel-Corrected Standard Errors, Cointegration Models & Error Correcting Models, etc. - I’m personally not well acquainted with most of the methods above: I had to look them up. Your job would be to understand what each of these methods is doing, when and why it should be used, and write up a section for the paper justifying the kind of method we’re using. - We must figure out how to model and account for imputation induced uncertainty in our regression estimates. Look into producing a “Stochastic SSPI” which “Rubin’s Rules” to account for within- and between-imputation uncertainty.


#### Question we are interested in understanding:

How does gdp affect STCONS/its sub-indicators over the years?

##### Why are we interested in examining the question?

To some extent, it highlights the importance of sustainability in the context of economic growth. It offers insights into how much of the environment may have been sacrificed in pursuit of economic development. This regression analysis will help us better understand the relationship between the two and provide a clearer picture of the intersection between GDP growth and sustainable development.

##### BASIC INFORMATION:
Indicator: Status Consumption \
Definition: Proportion of CO2 Emissions attributable to the Top 10% of earners. \
Datasets: \
Personal Carbon Footprint (Full Distribution) \
Personal Carbon Footprint (Top 10%) \
Ecological footprint Per Person 


##### STCONS Sub-Datasets Extracted
1. wid_carbon_tot_p0p100.csv \
Description: Total carbon emissions across all income groups \
Unit: tCO2 equivalent/cap (tonnes CO2 equivalent per capita) \
Records: 1,320 data points \
Coverage: Full income distribution (p0p100) 
2. wid_carbon_tot_p90p100.csv \
Description: Carbon emissions from the top 10% of earners \
Unit: tCO2 equivalent/cap (tonnes CO2 equivalent per capita) \
Records: 1,320 data points \
Coverage: Top 10% income percentiles (p90p100) 
3. fpi_ecofpt_per_cap.csv \
Description: Ecological footprint per capita \
Unit: gha per capita (global hectares per capita) \
Records: 3,989 data points \
Coverage: Ecological footprint of consumption
##### GDP (US $ World Bank):
https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2019&locations=1W&start=2000 

#### METHODOLOGY

We plan to employ two econometric approaches to analyze the relationship between environmental outcomes and economic activity, each addressing different aspects of the data structure and potential econometric challenges.

**1. Fixed Effects with Robust Standard Errors**

*What:* \
Fixed effects models control for unobserved time-invariant country characteristics (α_i) and common time shocks (γ_t) through dummy variables, while robust standard errors account for heteroskedasticity and correlation patterns.

*When:* \
We can use the model when countries have persistent unobserved characteristics that affect both GDP and environmental outcomes, and when temporal patterns suggest common shocks across countries.

*Why:* \
It is essential for causal identification by exploiting within-country variation over time, eliminating bias from omitted variables like geography, culture, or institutional quality.
- Clustered standard errors (by country) handle serial correlation within countries, while Driscoll-Kraay standard errors additionally account for cross-sectional dependence when countries face common shocks (e.g., global commodity prices, climate events). Moreover, Panel-Corrected Standard Errors assume AR(1) serial correlation within countries and use the Prais-Winsten transformation to correct for both serial correlation and cross-sectional dependence, providing more precise estimates when the AR(1) assumption is valid. Lastly, HAC corrects for heteroskedasticity and serial correlation over time within a panel, but does not account for cross-sectional dependence.

Some concepts clarification:
AR(1) assumption: It means that the values of a variable at different time points are correlated. In this case, the current value depends only on the previous value.
Prais-Winsten transformation:
If the error term follows AR(1), then the OLS estimate is unbiased but inefficient, and the standard errors are wrong. We need to transform the regression so that the errors become uncorrelated before estimating.
Here, we do not adopt it since it needs an R package.

**Regression:**
$$
y_t = \beta_0 + \beta_1 x_t + u_t
$$

**AR(1) error process:**
$$
u_t = \rho u_{t-1} + e_t
$$

**Prais–Winsten transform:**
$$
y_t - \rho y_{t-1} = \beta_0 (1 - \rho) + \beta_1 (x_t - \rho x_{t-1}) + e_t
$$

**Adjusted first observation (t = 1):**
$$
\sqrt{1 - \rho^2}\, y_1 = \sqrt{1 - \rho^2}\, \beta_0 + \sqrt{1 - \rho^2}\, \beta_1 x_1 + e_1
$$

Potential unobserved time-invariant country characteristics: 
1. weather and geographical location;
2. policy and culture;
3. international rules.

**2. Arellano-Bond Dynamic Panel GMM**

*What:* \
It is a generalized method of moments estimator that uses lagged levels and differences as instruments to address endogeneity and dynamic effects in panel data with short time dimensions.

*When:* \
We can apply the model when the dependent variable exhibits persistence (current values depend on past values) and when explanatory variables may be endogenous due to reverse causality or measurement error.

*Why:* \
It is critical for our analysis because environmental outcomes likely depend on their own history (path dependence), and GDP-environment relationships may be bidirectional. The method provides consistent estimates even when traditional fixed effects fail due to the "dynamic panel bias" problem.

**3. Cointegration and Error Correction Model**

*What:* \
This is a model that distinguishes between short-run dynamics and long-run equilibrium relationships among non-stationary variables, with error correction terms capturing adjustment toward equilibrium.

*When:* \
We can use it when variables are non-stationary but cointegrated, suggesting a stable long-run relationship despite short-run fluctuations.

*Why:* \
It can be particularly relevant for GDP-environment relationships, which may exhibit long-run equilibria but short-run deviations due to policy changes, technological shocks, or business cycles. This approach provides richer policy insights by separating immediate impacts from long-term adjustments.

### Indicator System 

#### DATA Loading

In [247]:
gdp_raw =pd.read_csv("m2.csv", skiprows =4)
gdp_raw.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09,3.648573e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.420993e+10,2.496326e+10,2.707802e+10,3.177483e+10,3.028492e+10,3.381219e+10,...,8.289612e+11,9.730251e+11,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12,1.176910e+12,1.287677e+12,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10,1.715223e+10,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.190511e+10,1.270803e+10,1.363092e+10,1.446926e+10,1.580394e+10,1.692124e+10,...,7.000282e+11,6.940513e+11,7.778404e+11,8.332889e+11,7.972952e+11,8.581145e+11,8.936399e+11,8.147285e+11,6.700257e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11,8.487516e+10,8.039694e+10,NaN


In [248]:
stcons_data = pd.read_csv('stcons_data.csv', header= None)
col_names = ['Country Code', 'Year','Score','Index']
stcons_data.columns = col_names
stcons_data = stcons_data.drop('Index',axis=1)
stcons_data = stcons_data.rename(columns={'Country Code':'CountryCode','Score':'Value'})
stcons_data

,CountryCode,Year,Value
0,ARG,2000,0.722311
1,ARG,2001,0.724883
2,ARG,2002,0.742501
3,ARG,2003,0.735742
4,ARG,2004,0.754505
...,...,...,...
1235,VNM,2015,0.821195
1236,VNM,2016,0.801584
1237,VNM,2017,0.791593
1238,VNM,2018,0.767574


In [249]:
fpi_eco = pd.read_csv('fpi_ecofpt_per_cap.csv')
fpi_eco = fpi_eco.drop('Unit',axis =1)
fpi_eco

,CountryCode,Year,Value
0,AFG,2000,0.805014
1,AFG,2001,0.833812
2,AFG,2002,0.985967
3,AFG,2003,0.887480
4,AFG,2004,0.782223
...,...,...,...
3984,ZWE,2020,1.203193
3985,ZWE,2021,1.252113
3986,ZWE,2022,1.222122
3987,ZWE,2023,1.209252


In [250]:
wid_10 = pd.read_csv('wid_carbon_tot_p90p100.csv')
wid_10 = wid_10.drop('Unit',axis =1)
wid_10

,CountryCode,Year,Value
0,ARE,2000,137.076599
1,ARE,2001,131.944962
2,ARE,2002,120.368217
3,ARE,2003,144.837936
4,ARE,2004,148.594543
...,...,...,...
1315,ZAF,2015,27.704645
1316,ZAF,2016,27.184013
1317,ZAF,2017,27.681314
1318,ZAF,2018,27.800943


In [251]:
wid_0 = pd.read_csv('wid_carbon_tot_p0p100.csv')
wid_0 = wid_0.drop('Unit',axis =1)
wid_0

,CountryCode,Year,Value
0,ARE,2000,43.553768
1,ARE,2001,41.106377
2,ARE,2002,37.337402
3,ARE,2003,44.517002
4,ARE,2004,45.494694
...,...,...,...
1315,ZAF,2015,7.243975
1316,ZAF,2016,7.114271
1317,ZAF,2017,7.235501
1318,ZAF,2018,7.229954


##### Data Basic Information Check:

In [252]:
country_name_chosen = wid_0['CountryCode']
#Number of unique countries
len(country_name_chosen.unique())

66

We observe 66 unique countries over 20 years.

In [323]:
#check non-positive values
print("stcons_data", (stcons_data['Value'] <= 0).sum(), "non-positive values")
print("fpi_eco", (fpi_eco['Value'] <= 0).sum(), "non-positive values")
print("wid_0", (wid_0['Value'] <= 0).sum(), "non-positive values")
print("wid_10", (wid_10['Value'] <= 0).sum(), "non-positive values")

#check null
print(stcons_data.isnull().sum())
print(fpi_eco.isnull().sum())
print(wid_0.isnull().sum())
print(wid_10.isnull().sum())

#check duplicates
print(stcons_data.duplicated().sum())
print(fpi_eco.duplicated().sum())
print(wid_0.duplicated().sum())
print(wid_10.duplicated().sum())

stcons_data 0 non-positive values
fpi_eco 0 non-positive values
wid_0 0 non-positive values
wid_10 0 non-positive values
CountryCode    0
Year           0
Value          0
dtype: int64
CountryCode    0
Year           0
Value          0
dtype: int64
CountryCode    0
Year           0
Value          0
dtype: int64
CountryCode    0
Year           0
Value          0
dtype: int64
0
0
0
0


#### Data Cleaning

In [255]:
#add a small values to those 0 values
stcons_data.loc[stcons_data['Value'] == 0, 'Value'] += 1e-6

In [256]:
df_selected = pd.concat([gdp_raw.iloc[:, :2], gdp_raw.iloc[:, 44:64]], axis=1)
#standardize
df_selected = df_selected.rename(columns={'Country Code': 'CountryCode'})
df_selected.iloc[:, 2:] = df_selected.iloc[:, 2:] / 1_000_000
df_selected

,Country Name,CountryCode,2000,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,1873.452514,1896.456983,1961.843575,2044.111732,2254.830726,2360.017318,2469.782682,2677.641341,...,2453.597207,2637.859218,2615.208380,2727.849721,2790.849721,2962.907263,2983.635196,3092.429050,3.276184e+03,3.395799e+03
1,Africa Eastern and Southern,AFE,287201.651521,260992.228316,267815.037640,355716.369818,442696.186423,516661.066865,580240.750536,665598.701334,...,849409.606775,945439.030450,952998.471719,962441.293816,978743.758351,898308.853395,828961.170566,973025.126474,1.012291e+06,1.009747e+06
2,Afghanistan,AFG,3521.418060,2813.571754,3825.701439,4520.946819,5224.896719,6203.256539,6971.758282,9747.886187,...,15856.668556,17805.098206,19907.329778,20146.416758,20497.128556,19134.221645,18116.572395,18753.456498,1.805322e+04,1.879944e+04
3,Africa Western and Central,AFW,142140.075688,150058.483261,179390.065281,207754.986323,258566.656911,317096.476047,402724.221239,471537.814859,...,606280.114371,691187.455140,748126.849687,844202.569831,903933.677160,778022.060797,700028.171710,694051.344215,7.778404e+05,8.332889e+05
4,Angola,AGO,9129.594970,8936.079118,15285.592370,17812.704586,23552.057679,36970.900884,52381.025141,65266.415494,...,83799.474070,111789.747671,128052.915766,132339.109040,135966.802587,90496.420507,52761.617226,73690.154991,7.945069e+04,7.089796e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5343.950556,6341.613610,6163.484245,6735.328610,7074.394735,6295.848423,6682.677290,7180.764703,7.878760e+03,7.899738e+03
262,"Yemen, Rep.",YEM,9679.316770,9852.990693,10693.430511,11777.532662,13867.634371,16731.566717,19063.143370,21650.528674,...,30906.749533,32726.417878,35401.331609,40415.233436,43228.585321,42444.490074,31317.825274,26842.229045,2.160616e+04,NaN
263,South Africa,ZAF,151752.757215,135429.905923,129087.556612,197018.965309,255806.908595,288867.217197,303858.675364,333077.117254,...,417363.822802,458199.494831,434400.545086,400886.013596,381198.869776,346709.790459,323585.509674,381448.814653,4.052607e+05,3.893300e+05
264,Zambia,ZMB,3600.632111,4094.441301,4193.850445,4901.869764,6221.110219,8331.870169,12756.858899,14056.957976,...,20265.559484,23459.515276,25503.060420,28037.239463,27141.023558,21251.216799,20958.412538,25873.601261,2.631151e+04,2.330867e+04


In [257]:
contained_null = df_selected.loc[df_selected.isnull().any(axis=1), 'CountryCode'].unique()
contained_null

array(['ASM', 'CHI', 'CYM', 'ERI', 'GIB', 'GUM', 'INX', 'MAF', 'MNP',
       'PRK', 'SSD', 'SXM', 'TCA', 'VEN', 'VGB', 'VIR', 'XKX', 'YEM'],
      dtype=object)

In [258]:
df_selected = df_selected[~df_selected['CountryCode'].isin(contained_null)]
df_selected

,Country Name,CountryCode,2000,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,1.873453e+03,1.896457e+03,1.961844e+03,2.044112e+03,2.254831e+03,2.360017e+03,2.469783e+03,2.677641e+03,...,2.453597e+03,2.637859e+03,2.615208e+03,2.727850e+03,2.790850e+03,2.962907e+03,2.983635e+03,3.092429e+03,3.276184e+03,3.395799e+03
1,Africa Eastern and Southern,AFE,2.872017e+05,2.609922e+05,2.678150e+05,3.557164e+05,4.426962e+05,5.166611e+05,5.802408e+05,6.655987e+05,...,8.494096e+05,9.454390e+05,9.529985e+05,9.624413e+05,9.787438e+05,8.983089e+05,8.289612e+05,9.730251e+05,1.012291e+06,1.009747e+06
2,Afghanistan,AFG,3.521418e+03,2.813572e+03,3.825701e+03,4.520947e+03,5.224897e+03,6.203257e+03,6.971758e+03,9.747886e+03,...,1.585667e+04,1.780510e+04,1.990733e+04,2.014642e+04,2.049713e+04,1.913422e+04,1.811657e+04,1.875346e+04,1.805322e+04,1.879944e+04
3,Africa Western and Central,AFW,1.421401e+05,1.500585e+05,1.793901e+05,2.077550e+05,2.585667e+05,3.170965e+05,4.027242e+05,4.715378e+05,...,6.062801e+05,6.911875e+05,7.481268e+05,8.442026e+05,9.039337e+05,7.780221e+05,7.000282e+05,6.940513e+05,7.778404e+05,8.332889e+05
4,Angola,AGO,9.129595e+03,8.936079e+03,1.528559e+04,1.781270e+04,2.355206e+04,3.697090e+04,5.238103e+04,6.526642e+04,...,8.379947e+04,1.117897e+05,1.280529e+05,1.323391e+05,1.359668e+05,9.049642e+04,5.276162e+04,7.369015e+04,7.945069e+04,7.089796e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,World,WLD,3.386871e+07,3.366229e+07,3.495994e+07,3.920559e+07,4.418190e+07,4.784764e+07,5.185967e+07,5.844861e+07,...,6.672215e+07,7.421304e+07,7.586601e+07,7.807009e+07,8.024743e+07,7.572364e+07,7.695735e+07,8.197480e+07,8.718899e+07,8.849239e+07
260,Samoa,WSM,2.588561e+02,2.662996e+02,2.817901e+02,3.334262e+02,4.077476e+02,4.768018e+02,4.999238e+02,5.735485e+02,...,6.802609e+02,7.440971e+02,7.731417e+02,7.977363e+02,7.966835e+02,8.241505e+02,8.439248e+02,8.848444e+02,8.784484e+02,9.129505e+02
263,South Africa,ZAF,1.517528e+05,1.354299e+05,1.290876e+05,1.970190e+05,2.558069e+05,2.888672e+05,3.038587e+05,3.330771e+05,...,4.173638e+05,4.581995e+05,4.344005e+05,4.008860e+05,3.811989e+05,3.467098e+05,3.235855e+05,3.814488e+05,4.052607e+05,3.893300e+05
264,Zambia,ZMB,3.600632e+03,4.094441e+03,4.193850e+03,4.901870e+03,6.221110e+03,8.331870e+03,1.275686e+04,1.405696e+04,...,2.026556e+04,2.345952e+04,2.550306e+04,2.803724e+04,2.714102e+04,2.125122e+04,2.095841e+04,2.587360e+04,2.631151e+04,2.330867e+04


In [259]:
df_long = df_selected.melt(
    id_vars=['Country Name', 'CountryCode'],  
    var_name='Year',
    value_name='Value'
)

df_long['Year'] = df_long['Year'].astype(int)
df_long = df_long.sort_values(by=['CountryCode', 'Year']).reset_index(drop=True)

df_chosen = df_long[df_long['CountryCode'].isin(country_name_chosen)]
df_chosen

,Country Name,CountryCode,Year,Value
160,United Arab Emirates,ARE,2000,104337.372362
161,United Arab Emirates,ARE,2001,103311.640572
162,United Arab Emirates,ARE,2002,109816.201498
163,United Arab Emirates,ARE,2003,124346.358067
164,United Arab Emirates,ARE,2004,147824.370320
...,...,...,...,...
4915,South Africa,ZAF,2015,346709.790459
4916,South Africa,ZAF,2016,323585.509674
4917,South Africa,ZAF,2017,381448.814653
4918,South Africa,ZAF,2018,405260.723893


In [325]:
years_gdp = set(df_chosen['Year'].unique())
years_wid0 = set(wid_0['Year'].unique())
years_wid10 = set(wid_10['Year'].unique())
years_fpi = set(fpi_eco['Year'].unique()) #find unique value
years_stcons = set(stcons_data['Year'].unique())

# find common years
common_years = sorted(list(years_gdp & years_wid0 & years_wid10 & years_fpi & years_stcons))

# filter to only contain those years
df_chosen_int = df_chosen[df_chosen['Year'].isin(common_years)].copy()
w0_int = wid_0[wid_0['Year'].isin(common_years)].copy()
w10_int = wid_10[wid_10['Year'].isin(common_years)].copy()
fpi_int = fpi_eco[fpi_eco['Year'].isin(common_years)].copy()
stcons_int = stcons_data[stcons_data['Year'].isin(common_years)].copy()

# merge 
panel_int = df_chosen_int.copy()
panel_int = panel_int.merge(w0_int[['CountryCode','Year','Value']], on=['CountryCode','Year'], how='left', suffixes=('', '_wid0'))
panel_int = panel_int.merge(w10_int[['CountryCode','Year','Value']], on=['CountryCode','Year'], how='left', suffixes=('', '_wid10'))
panel_int = panel_int.merge(fpi_int[['CountryCode','Year','Value']], on=['CountryCode','Year'], how='left', suffixes=('', '_fpi'))
panel_int = panel_int.merge(stcons_int[['CountryCode','Year','Value']], on=['CountryCode','Year'], how='left', suffixes=('', '_stcons'))
panel_int = panel_int.dropna()

In [261]:
panel = panel_int.reset_index().drop('index',axis=1)
panel

,Country Name,CountryCode,Year,Value,Value_wid0,Value_wid10,Value_fpi,Value_stcons
0,Argentina,ARG,2000,284203.750000,7.326881,22.097914,3.145340,0.722311
1,Argentina,ARG,2001,268696.750000,7.193967,21.654575,3.127239,0.724883
2,Argentina,ARG,2002,97724.004252,5.395169,16.224895,2.963778,0.742501
3,Argentina,ARG,2003,127586.973492,6.235285,17.815453,3.186665,0.735742
4,Argentina,ARG,2004,164657.930453,6.659634,18.383343,3.105351,0.754505
...,...,...,...,...,...,...,...,...
1215,South Africa,ZAF,2015,346709.790459,7.243975,27.704645,3.287789,0.613585
1216,South Africa,ZAF,2016,323585.509674,7.114271,27.184013,3.222784,0.622731
1217,South Africa,ZAF,2017,381448.814653,7.235501,27.681314,3.402535,0.597983
1218,South Africa,ZAF,2018,405260.723893,7.229954,27.800943,3.335963,0.604662


#### EDA

In [263]:
fig = go.Figure()

# add lines for each country
for code, sub in wid_0.groupby('CountryCode'):
    fig.add_trace(go.Scatter(
        x=sub['Year'], 
        y=sub['Value'],
        mode='lines+markers',
        name=code,
        opacity=0.7,
        hovertemplate=f'<b>{code}</b><br>' +
                     'Year: %{x}<br>' +
                     'Value: %{y:.2f}<extra></extra>'
    ))

fig.update_layout(
    title='Interactive Carbon Emissions Over Time',
    xaxis_title='Year',
    yaxis_title='tCO2 equivalent/cap',
    hovermode='closest',
    width=1000,
    height=600
)

fig.show()

Countries vary greatly in their CO₂ emissions, with some exceeding 40 tons per capita. While certain countries show relatively stable emission trajectories, others experience substantial fluctuations over time.

In [265]:
fig2 = go.Figure()
for code, sub in wid_10.groupby('CountryCode'):
    fig2.add_trace(go.Scatter(
        x=sub['Year'], 
        y=sub['Value'],
        mode='lines+markers',
        name=code,
        opacity=0.7,
        hovertemplate=f'<b>{code}</b><br>' +
                     'Year: %{x}<br>' +
                     'Value: %{y:.2f}<extra></extra>'
    ))

fig2.update_layout(
    title='Interactive Carbon Emissions from Top 10% Earners Over Time',
    xaxis_title='Year',
    yaxis_title='tCO2 equivalent/cap',
    hovermode='closest',
    width=1000,
    height=600
)

fig2.show()

Compared with the previous figure, the top 10% of earners show a wider range, but their overall distribution is broadly similar to that of per-capita carbon emissions.

In [267]:
fig3 = go.Figure()
for code, sub in fpi_eco.groupby('CountryCode'):
    fig3.add_trace(go.Scatter(
        x=sub['Year'], 
        y=sub['Value'],
        mode='lines+markers',
        name=code,
        opacity=0.7,
        hovertemplate=f'<b>{code}</b><br>' +
                     'Year: %{x}<br>' +
                     'Value: %{y:.2f}<extra></extra>'
    ))

fig3.update_layout(
    title='Ecological footprint per capita',
    xaxis_title='Year',
    yaxis_title='gha per capita',
    hovermode='closest',
    width=1000,
    height=600
)

fig3.show()

It is noteworthy that Luxembourg displays the highest carbon footprint trajectory. Possible explanations include its small population, extensive cross-border mobility, and the characteristics associated with a high-income economy.

In [269]:
fig4 = go.Figure()
for code, sub in df_chosen.groupby('CountryCode'):
    fig4.add_trace(go.Scatter(
        x=sub['Year'], 
        y=sub['Value'],
        mode='lines+markers',
        name=code,
        opacity=0.7,
        hovertemplate=f'<b>{code}</b><br>' +
                     'Year: %{x}<br>' +
                     'Value: %{y:.2f}<extra></extra>'
    ))

fig4.update_layout(
    title='GDP',
    xaxis_title='Year',
    yaxis_title='$ in millions',
    hovermode='closest',
    width=1000,
    height=600
)

fig4.show()

From 2000 to 2019, the USA and China show strong upward trends, with the USA’s ecological footprint consistently about twice that of China’s. The difference is shortening. Moreover, Japan ranks third with small fluctuation, while the remaining countries remain stable, each staying below 5 million gha per capita throughout the 20 years.

In [271]:
fig5 = go.Figure()
for code, sub in stcons_data.groupby('CountryCode'):
    fig5.add_trace(go.Scatter(
        x=sub['Year'], 
        y=sub['Value'],
        mode='lines+markers',
        name=code,
        opacity=0.7,
        hovertemplate=f'<b>{code}</b><br>' +
                     'Year: %{x}<br>' +
                     'Value: %{y:.2f}<extra></extra>'
    ))

fig5.update_layout(
    title='Status Consumption',
    xaxis_title='Year',
    yaxis_title='Score',
    hovermode='closest',
    width=1000,
    height=600
)

fig5.show()

- From the figure, we observe that Bangladesh ranks highest, although its score has declined over the years. The top-ranked countries show relatively small fluctuations, while those in the middle and lower ranks fluctuate much more.
- Singapore has remained 0 for many years.
- Developed countries tend to score low.

#### Model Building:

Fixed Effects is appropriate for our data because:
- Data Structure: Panel data with large cross-country differences
- Theoretical Support: Unobserved heterogeneity theory
- Statistical Tests: Results are robust across methods (which will be addressed afterwards)
- Policy Relevance: Provides causal interpretation
- Methodological Soundness: Addresses endogeneity concerns

##### FE Model:

$$
\log(\text{wid0})_{it}
= \beta \log(\text{gdp})_{it}
+ \alpha_i \;(\text{entity\;FE})
+ \lambda_t \;(\text{time\;FE})
+ \varepsilon_{it}
$$

- $\alpha_i$: entity fixed effect (unit-specific effect)  
- $\lambda_t$: time fixed effect (year-specific effect)  
- $\varepsilon_{it}$: error term (unobserved random variation not captured by the model)
- ${i}$: 1 - 66
- ${t}$: 1 - 19

I did not impute data since several countries have 0 values for all studied years.

In [278]:
# data_for_impute = panel_wide.copy()
# # MICE over numeric columns only
# num_cols = ['gdp','Value_wid0','Value_wid10','Value_fpi','Value_stcons']
# imp = IterativeImputer(random_state=42, max_iter=20, sample_posterior=True)

# m = 5 
# for k in range(m):
#     imputer = IterativeImputer(random_state=42 + k, max_iter=20, sample_posterior=True)
#     imputed_array = imputer.fit_transform(data_for_impute[num_cols])
#     imputed = data_for_impute.copy()
#     imputed[num_cols] = imputed_array
#     # FE using imputed data
#     df_imp = imputed.dropna(subset=['CountryCode','Year']).copy()
#     df_imp = df_imp.set_index(['CountryCode','Year'])
#     df_imp['log_gdp'] = np.log(df_imp['gdp'])
#     df_imp['log_wid0'] = np.log(df_imp['Value_wid0'])
#     df_imp['log_wid10'] = np.log(df_imp['Value_wid10'])
#     df_imp['log_fpi'] = np.log(df_imp['Value_fpi'])
#     df_imp['log_stcons'] = np.log(df_imp['Value_stcons'])

panel2 = panel[['CountryCode','Year','Value_wid0','Value_wid10','Value_fpi', 'Value_stcons','Value']].rename(columns={'Value':'gdp'})
df_imp = panel2.set_index(['CountryCode','Year'])
df_imp['log_gdp'] = np.log(df_imp['gdp'])
df_imp['log_wid0'] = np.log(df_imp['Value_wid0'])
df_imp['log_wid10'] = np.log(df_imp['Value_wid10'])
df_imp['log_fpi'] = np.log(df_imp['Value_fpi'])
df_imp['log_stcons'] = np.log(df_imp['Value_stcons'])


#model: log_wid0 ~ log_gdp + FE (Entity + Time) 
results_kernel, results_dk, results_cse = {}, {}, {}

for dep in ['log_wid0','log_wid10','log_fpi','log_stcons']:
    model = PanelOLS.from_formula(f"{dep} ~ log_gdp + EntityEffects + TimeEffects", data=df_imp)
    
    # HAC
    fit = model.fit(cov_type='kernel')
    results_kernel[dep] = {
        'beta': fit.params['log_gdp'],
        'se': fit.std_errors['log_gdp']
    }
    
    # Driscoll–Kraay 
    fit_dk = model.fit(cov_type='driscoll-kraay')
    results_dk[dep] = {
        'beta': fit_dk.params['log_gdp'],
        'se': fit_dk.std_errors['log_gdp']
    }
    
    # Clustered SE
    fit_cse = model.fit(cov_type='clustered', cluster_entity=True)
    results_cse[dep] = {
        'beta': fit_cse.params['log_gdp'],
        'se': fit_cse.std_errors['log_gdp']
    }

    
#Rubin's rules combine
# def rubin_pool(estimates):
#     # estimates: list of dicts with {'beta': float, 'se': float}
#     m_ = len(estimates)
#     betas = np.array([e['beta'] for e in estimates], dtype=float)
#     ses = np.array([e['se'] for e in estimates], dtype=float)
#     q_bar = betas.mean()
#     u_bar = np.mean(ses**2)
#     b_m = betas.var(ddof=1)
#     t_var = u_bar + (1 + 1/m_) * b_m
#     t_se = math.sqrt(t_var)
#     return q_bar, t_se

In [279]:
#print(fit.summary)

In [327]:
all_results = []

for dep in ['log_wid0','log_wid10','log_fpi','log_stcons']:
    for method, results in {
        "HAC": results_kernel,
        "Driscoll–Kraay": results_dk,
        "Clustered SE": results_cse}.items():
        beta = results[dep]['beta']
        se   = results[dep]['se']
        tval = beta / se
        pval = 2 * (1 - stats.norm.cdf(abs(tval)))  #normal
        ci_low, ci_high = beta - 1.96*se, beta + 1.96*se

        all_results.append({
            'dependent_var': dep,
            'method': method,
            'coef': round(beta,5),
            'se': round(se,5),
            't': round(tval,5),
            'pval': round(pval,5),
            'ci_low': round(ci_low,5),
            'ci_high': round(ci_high,5),
            'significant_5pct': pval < 0.05
        })

df_results = pd.DataFrame(all_results)
df_results = df_results[['dependent_var','method','coef','se','t','pval','ci_low','ci_high','significant_5pct']]

df_results

,dependent_var,method,coef,se,t,pval,ci_low,ci_high,significant_5pct
0,log_wid0,HAC,0.42977,0.01719,25.00487,0.00000,0.39608,0.46346,True
1,log_wid0,Driscoll–Kraay,0.42977,0.01719,25.00487,0.00000,0.39608,0.46346,True
2,log_wid0,Clustered SE,0.42977,0.03881,11.07443,0.00000,0.35371,0.50583,True
3,log_wid10,HAC,0.43870,0.01524,28.77859,0.00000,0.40882,0.46858,True
4,log_wid10,Driscoll–Kraay,0.43870,0.01524,28.77859,0.00000,0.40882,0.46858,True
5,log_wid10,Clustered SE,0.43870,0.05001,8.77242,0.00000,0.34068,0.53672,True
6,log_fpi,HAC,0.29713,0.01544,19.24909,0.00000,0.26687,0.32738,True
7,log_fpi,Driscoll–Kraay,0.29713,0.01544,19.24909,0.00000,0.26687,0.32738,True
8,log_fpi,Clustered SE,0.29713,0.04777,6.21989,0.00000,0.20350,0.39076,True
9,log_stcons,HAC,-0.68020,0.12101,-5.62119,0.00000,-0.91737,-0.44302,True


Interpretation:
- A 1% increase in GDP is associated with a 43% rise in the overall population’s carbon footprint, a 43.8% increase in the carbon footprint of the wealthiest 10%, and a 29.7% increase in the ecological footprint.
- In contrast, status consumption declines by 68%, suggesting that as GDP grows, inequality in elite consumption may diminish.
- Since all p-values are below 0.05, the results are statistically significant across all parameters.
- Among the three covariance estimators, clustered standard errors are the smallest. This is expected because clustering accounts only for within-country serial correlation and heteroskedasticity, but assumes independence across countries. In contrast, Driscoll–Kraay errors are larger, as they also adjust for cross-sectional dependence due to common global shocks. The similarity of results across methods, however, suggests that our main findings are robust.

##### Arellano-Bond GMM

Arellano-Bond Model

$$
y_{it} = \alpha + \beta\, y_{i,t-1} + \gamma\, x_{it} + \mu_i + \varepsilon_{it}
$$

- $y_{it}$: Dependent variable for unit $i$ at time $t$.
- $y_{i,t-1}$: One-period lag of the dependent variable for unit $i$ (captures dynamics/persistence).
- $x_{it}$: Covariate(s) for unit $i$ at time $t$. If $x_{it}$ is a vector, write $\gamma^\top x_{it}$.
- $\alpha$: Common intercept (constant term).
- $\beta$: Coefficient on the lagged dependent variable (degree of persistence).
- $\gamma$: Coefficient (vector) on $x_{it}$ (marginal effect of $x_{it}$ on $y_{it}$).
- $\mu_i$: Unit fixed effect (time-invariant unobserved heterogeneity for unit $i$).
- $\varepsilon_{it}$: Error term.

In [340]:
def prepare_gmm_data(df_base, dep_var, exog_vars, min_periods=3):   
    #+lagged variables
    df_gmm = df_base.copy()
    df_gmm[f'{dep_var}_lag1'] = df_gmm.groupby('CountryCode')[dep_var].shift(1)
    df_gmm[f'{dep_var}_lag2'] = df_gmm.groupby('CountryCode')[dep_var].shift(2)
    df_gmm[f'{dep_var}_lag3'] = df_gmm.groupby('CountryCode')[dep_var].shift(3)
    
    #create differenced variables (may or may not be useful)
    df_gmm[f'{dep_var}_diff'] = df_gmm.groupby('CountryCode')[dep_var].diff()
    df_gmm[f'{dep_var}_lag1_diff'] = df_gmm.groupby('CountryCode')[f'{dep_var}_lag1'].diff()
    
    #+exogenous variables
    for exog_var in exog_vars:
        df_gmm[f'{exog_var}_lag1'] = df_gmm.groupby('CountryCode')[exog_var].shift(1)
        df_gmm[f'{exog_var}_lag2'] = df_gmm.groupby('CountryCode')[exog_var].shift(2)
        df_gmm = df_gmm.dropna()     #if any missing
    
    #filter countries with sufficient observations
    country_counts = df_gmm.groupby('CountryCode').size()
    valid_countries = country_counts[country_counts >= min_periods].index
    df_gmm = df_gmm[df_gmm.index.get_level_values('CountryCode').isin(valid_countries)]
    
    print(f"Number of countries: {df_gmm.index.get_level_values('CountryCode').nunique()}")
    print(f"Total observations: {len(df_gmm)}")
    print(f"Average observations per country: {len(df_gmm) / df_gmm.index.get_level_values('CountryCode').nunique():.1f}")    
    return df_gmm

In [342]:
#Analysis and coding

In [337]:
#Comparison between two methods.

##### Cointegration and Error Correction Models

All variables will change over time.
Since we have insufficient non-stationary variables, we won't adopt this method.

#### Conclusion:

In [ ]:
main discovery:
From the 
policy meaning
acknowledge
future direction

#### References: 

1. Princeton Panel Data Analysis: https://www.princeton.edu/~otorres/Panel101.pdf;
2. http://fmwww.bc.edu/EC-C/S2013/823/EC823.S2013.nn05.slides.pdf;
3. https://support.sas.com/resources/papers/proceedings17/SAS0642-2017.pdf;
4. https://benjamindwilliams.weebly.com/uploads/6/8/5/7/68575765/lecture8_revised.pdf?utm_source=chatgpt.com;